In [ ]:
!pip install ultralytics roboflow --quiet

from roboflow import Roboflow
from ultralytics import YOLO
from google.colab import drive
import os
import yaml
import glob


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROBOFLOW_API_KEY = ""
WORKSPACE_ID = ""
PROJECT_ID = "indonesia-walkability"
VERSION_NUMBER = 1
CLASSES_TO_KEEP = ["road", "sidewalk", "tree"]


In [ ]:
def filter_and_remap_labels(dataset_location, classes_to_keep):
    data_yaml_path = os.path.join(dataset_location, "data.yaml")
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    original_names = data['names']
    old_idx_to_new_idx = {}
    new_names = []
    for i, name in enumerate(original_names):
        if name in classes_to_keep:
            new_idx = len(new_names)
            old_idx_to_new_idx[i] = new_idx
            new_names.append(name)
    print(f"{old_idx_to_new_idx}")

    data['names'] = new_names
    data['nc'] = len(new_names)
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    print(f"{new_names}")

    for split in ['train', 'valid', 'test']:
        label_dir = os.path.join(dataset_location, split, 'labels')
        if not os.path.exists(label_dir):
            continue

        print(f"{split}")
        for label_file in glob.glob(os.path.join(label_dir, '*.txt')):
            with open(label_file, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                old_class_idx = int(parts[0])

                if old_class_idx in old_idx_to_new_idx:
                    new_class_idx = old_idx_to_new_idx[old_class_idx]
                    new_line = f"{new_class_idx} {' '.join(parts[1:])}"
                    new_lines.append(new_line)

            with open(label_file, 'w') as f:
                f.write('\n'.join(new_lines))

In [ ]:
try:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(WORKSPACE_ID).project(PROJECT_ID)
    dataset = project.version(VERSION_NUMBER).download("yolov11")
    DATASET_LOCATION = dataset.location
    filter_and_remap_labels(DATASET_LOCATION, CLASSES_TO_KEEP)

except Exception as e:
    exit()

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Indonesia-Walkability-1 in yolov8:: 100%|██████████| 4346/4346 [00:00<00:00, 5094.54it/s]


{3: 0, 4: 1, 5: 2}
['road', 'sidewalk', 'tree']
train
valid
test


In [ ]:
model = YOLO('yolo11s-seg.pt')

try:
    results = model.train(
        data=os.path.join(DATASET_LOCATION, "data.yaml"),
        epochs=150,
        patience=20,
        batch=16,
        imgsz=640,
        project="/content/drive/MyDrive/YOLO11_Training_Results",
        name="walkability_yolo11_segmentation"
    )
    print("Training Success!")

except Exception as e:
    print(f"There is an error!. Detail: {e}")

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Indonesia-Walkability-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=walkability_yolo11_segmentation, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspe

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

 23        [16, 19, 22]  1   1475065  ultralytics.nn.modules.head.Segment          [3, 32, 128, [128, 256, 512]] 
YOLO11s-seg summary: 203 layers, 10,083,449 parameters, 10,083,433 gradients, 35.6 GFLOPs

Transferred 555/561 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1765.0±814.4 MB/s, size: 64.6 KB)


train: Scanning /content/Indonesia-Walkability-1/train/labels... 1896 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1896/1896 [00:00<00:00, 1953.16it/s]

...

    122/150      7.03G     0.4865     0.8352     0.3701     0.9679         29        640: 100%|██████████| 119/119 [00:54<00:00,  2.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.46it/s]

                   all        181        312      0.623      0.538      0.529      0.411      0.651      0.545      0.557      0.368
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 102, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



122 epochs completed in 1.957 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO11_Training_Results/walkability_yolo11_segmentation/weights/last.pt, 20.5MB
Optimizer stripped from /content/drive/MyDrive/YOLO11_Training_Results/walkability_yolo11_segmentation/weights/best.pt, 20.5MB

Validating /content/drive/MyDrive/YOLO11_Training_Results/walkability_yolo11_segmentation/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-seg summary (fused): 113 layers, 10,067,977 parameters, 0 gradients, 35.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:09<00:00,  1.57s/it]


                   all        181        312      0.645      0.551      0.556      0.407      0.681      0.573      0.584      0.375
                  road        172        192      0.871      0.906      0.926      0.791      0.878      0.911      0.922      0.767
              sidewalk         56         62      0.673      0.516      0.521      0.367      0.717      0.548      0.567       0.28
                  tree         28         58       0.39      0.232      0.221     0.0624      0.446      0.259      0.264     0.0788
Speed: 0.5ms preprocess, 7.6ms inference, 0.0ms loss, 15.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO11_Training_Results/walkability_yolo11_segmentation
Training Success!
